# **Tutorial Basic Spark RDD**

### Alif Mahardhika - 1706021934

In [1]:
'''
Karena saya menjalankan spark di machine windows pribadi, perlu ada konfigurasi imports tambahan:
'''
import findspark
SPARK_HOME='C:\spark\spark-3.0.1-bin-hadoop2.7'
findspark.init('C:/spark/spark-3.0.1-bin-hadoop2.7')
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

## Persiapan:

Pada bagian ini sebuah spark context dan session di inisiasi. Saya juga mendefinisikan variabel header untuk menyimpan nama-nama column berdasarkan interpretasi saya.

### Soal 2.1: Load data format ke RDD dan hitung jumlah user.

In [63]:
#Inisiasi spark session
spark = SparkSession \
.builder \
.appName("Tutorial Basic Spark") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()


#membaca file u.user sebagai csv
users = spark.read.csv(path = "ml-100k/u.user", 
                       sep = '|',
                       encoding = 'UTF-8',
                       comment = None ,
                       header = False ,
                       inferSchema = True)

users.rdd.take(5)

943

In [64]:
# Soal nomor 1: hitung jumlah user

print("Number of users: " +str(users.rdd.count()))



Number of users: 943


### Soal 2.2: Buat fungsi map yang membagi user ke rentang usia 10-80 tahun buat bin sebesar 10 tahun setiap kategori.

In [76]:
# buat fungsi klasifikasi age_group antara 10-80 dengan ukuran range (bin) 10
def classify_age_group(age):
    if age > 10 and age <= 20:
        return "10-20"
    elif age > 20 and age <= 30:
        return "20-30"
    elif age > 30 and age <= 40:
        return "30-40"
    elif age > 40 and age <= 50:
        return "40-50"
    elif age > 50 and age <= 60:
        return "50-60"
    elif age > 60 and age <= 70:
        return "60-70"
    elif age > 70 and age <= 80:
        return "70-80"
    elif age <= 10:
        return "under_age"
    else:
        return "over_age"

In [75]:
# buat header baru untuk menyimpan column age_group
header = ['user_id', 'age', 'gender', 'occupation','zipcode', 'age_group']

# map users sehingga memiliki age group yang sesuai. age_group akan didefine sebagai kolom baru
users_age_grouped = users.rdd.map(lambda x: [x['_c0'],x['_c1'],x['_c2'],x['_c3'],x['_c4'],classify_age_group(x['_c1'])])


users_age_grouped_df = users_age_grouped.toDF(header)
users_age_grouped_df.show()

+-------+---+------+-------------+-------+---------+
|user_id|age|gender|   occupation|zipcode|age_group|
+-------+---+------+-------------+-------+---------+
|      1| 24|     M|   technician|  85711|    20-30|
|      2| 53|     F|        other|  94043|    50-60|
|      3| 23|     M|       writer|  32067|    20-30|
|      4| 24|     M|   technician|  43537|    20-30|
|      5| 33|     F|        other|  15213|    30-40|
|      6| 42|     M|    executive|  98101|    40-50|
|      7| 57|     M|administrator|  91344|    50-60|
|      8| 36|     M|administrator|  05201|    30-40|
|      9| 29|     M|      student|  01002|    20-30|
|     10| 53|     M|       lawyer|  90703|    50-60|
|     11| 39|     F|        other|  30329|    30-40|
|     12| 28|     F|        other|  06405|    20-30|
|     13| 47|     M|     educator|  29206|    40-50|
|     14| 45|     M|    scientist|  55106|    40-50|
|     15| 49|     F|     educator|  97301|    40-50|
|     16| 21|     M|entertainment|  10309|    

### Soal 2.3: Hitung jumlah pengguna berdasarkan profesi mereka dalam age_group 20-30 yang diberikan.


In [82]:
from operator import add

# buat rdd baru dengan memfilter berdasarkan hasil klasifikasi rentang umur, lalu filter berdasarkan key (occupation) dan reduksi jika ada key sama dengan menambahkan
users_20_30_occupations = users.rdd.filter(lambda x: classify_age_group(x['_c1']) == "20-30").map(lambda x: (x[3], 1)).reduceByKey(add)

users_20_30_occupations.toDF(['occupation', 'num_of_users']).show()


+-------------+------------+
|   occupation|num_of_users|
+-------------+------------+
|   technician|          14|
|       writer|          16|
|      student|          94|
|        other|          42|
|entertainment|           8|
|   programmer|          34|
|       artist|          13|
|administrator|          24|
|    librarian|          15|
|    marketing|           7|
|     educator|          13|
|    executive|           7|
|     engineer|          25|
|     salesman|           4|
|       lawyer|           5|
|    scientist|           8|
|       doctor|           2|
|   healthcare|           4|
|         none|           2|
|    homemaker|           2|
+-------------+------------+



### Soal 2.4:  Hitung jumlah pengguna film dalam kelompok usia yang sama berdasarkan jenis kelamin

In [85]:
from pyspark.sql.functions import asc

users_gender_per_age_group = users_age_grouped_df.groupBy("age_group","gender").count().orderBy(asc('age_group'))

users_gender_per_age_group.show()

+---------+------+-----+
|age_group|gender|count|
+---------+------+-----+
|    10-20|     F|   42|
|    10-20|     M|   65|
|    20-30|     F|   87|
|    20-30|     M|  252|
|    30-40|     M|  162|
|    30-40|     F|   61|
|    40-50|     M|  109|
|    40-50|     F|   58|
|    50-60|     M|   60|
|    50-60|     F|   23|
|    60-70|     F|    2|
|    60-70|     M|   19|
|    70-80|     M|    1|
|under_age|     M|    2|
+---------+------+-----+



### Soal 2.5: Gunakan Akumulator untuk deteksi outlier dalam dataset film di atas. Asumsikan bahwa siapa saja yang termasuk dalam kelompok usia 80+ lebih sebagai outlier dan ditandai sebagai over_age dan siapa pun yang masuk dalam kelompok umur 0-10 juga outlier dan ditandai sebagai under_age

In [87]:
# Karena grouping outlier sudah dilakukan sebelumnya pada pengerjaan soal nomor2.2, pada bagian ini akan dilakukan penghitungan jumlah outlier menggunakan accumulator saja

sc = SparkContext.getOrCreate()

accum = sc.accumulator(0)
def count_outliers(row):
    global accum
    if(row['age_group'] == 'under_age' or row['age_group'] == 'over_age'):
        accum +=1
users_age_grouped_df.foreach(count_outliers)
print('Outlier Count: ' + str(accum.value))

Outlier Count: 2


In [ ]:
### Soal 2.6: Periksa berapa banyak pengguna di bawah usia dan berapa banyak di atas usia.

In [ ]:
outliers = users_age_grouped_df.filter("age_group <=10 or age >80")
outliers.show()